#**Modelo de Previsão de Fraude Financeira**

### **Introdução**

Conforme a era digital se expande, o uso de dinheiro físico se torna cada vez menos comum. Cartões físicos, virtuais, transferência por pix e pagamento através do celular são apenas alguns exemplos de soluções digitais para lidar com transações no mundo financeiro. Porém, com a abstração da ideia de possuir dinheiro, se torna mais fácil efetuar fraudes, como por acesso indevido a uma conta para transferir dinheiro para o agente malicioso que a invadiu, um problema fácil de ocorrer quando há descuido no armazenamento de senhas e nomes de usuário. Para evitar transtornos entre bancos e seus clientes, é ideal poder barrar qualquer tipo de transação que configure fraude, mesmo que em caso de falsos positivos, pois o prejuízo é menor.

Devido às dimensões massivas das transações financeiras digitais, é impossível lidar com essas previsões de maneira manual, o trabalho deve ser automatizado, de preferência de uma forma otimizada para rodar o mais rapidamente possível e não atrasar as transações ocorrendo em tempo real. Para tratar essa situação, propomos uma solução utilizando técnicas de inteligência artificial para classificar transações entre fraude e não fraude, num modelo altamente utilizado, chamado **árvore de decisão**.

### **Dados Utilizados**

Obter dados financeiros reais é um problema grande devido à violação de privacidade dos donos daqueles dados. Instituições financeiras dificilmente cederiam bases de dados para o público, portanto utilizamos de uma solução sintética, através da base de dados [Paysim](https://www.kaggle.com/ealaxi/paysim1), que, baseado em cerca de 96 milhões de transações financeiras dentro de 31 dias num serviço financeiro móvel de uma multinacional num país africano, gerou em torno de 24 milhões de transações artificiais, também ao longo de 31 dias, com classificação de "fraude" e "não fraude" para treinamento inteligências artificiais.

Cada transação possui 11 atributos que a definem. Abaixo descrevemos o que cada um significa.



1.   **step** - O momento (medido num intervalo de uma hora) em que a transação ocorre. Assume valores inteiros entre 1 e 744 (última hora do último dos 31 dias do mês).
2.   **type** - O tipo da transação, representado pelos seus nomes, que podem ser:
  *   CASH-IN: Depósito de dinheiro físico na conta.
  *   CASH-OUT: Saque de dinheiro físico da conta.
  *   DEBIT: Débito na conta.
  *   PAYMENT: Pagamento com a conta.
  *   TRANSFER: Transferência de uma conta para outra.
3.   **amount** - A quantidade de dinheiro na transação. Todo o banco de dados está numa mesma moeda não identificada. Os valores são números reais não-negativos.
4.   **nameOrig** - O "nome" da conta de origem da transação.
5.   **oldbalanceOrig** - A quantidade de dinheiro na conta de origem *antes* da transação ser realizada. Os valores são números reais não-negativos.
6.   **newbalanceOrig** - A quantidade de dinheiro na conta de origem *após* a transação ser realizada. Os valores são números reais não-negativos.
7.   **nameDest** - O "nome" da conta de destino da transação. 
8.   **oldbalanceDest** - A quantidade de dinheiro na conta de destino *antes* da transação ser realizada. Os valores são números reais não-negativos.
9.   **newbalanceDest** - A quantidade de dinheiro na conta de destino *após* a transação ser realizada. Os valores são números reais não-negativos.
10.  **isFraud** - Indica se a transação é fraude ou não. 0 para "não fraude" e 1 para "fraude".
11.  **isFlaggedFraud** - Indica se a transação já foi identificada como possível fraude. A regra para essa identificação é: qualquer transação com *amount >= 200000* é suspeita, caso contrário, não é. 0 para "não suspeita" e 1 para "suspeita".

Perceba que alguns desses campos não são necessários. Podemos definir *isFlaggedFraud* facilmente através de uma comparação *amount >= 200000*. *newbalanceOrig* e *newbalanceDest* podem ser definidos a partir de uma soma ou subtração de *amount* aos valores de *oldbalanceOrig* e *oldbalanceDest* de acordo com o *type* da transação. Esses atributos redundantes não agregam ao modelo e podem ser excluídos sem perdas de informação.

O campo *step* será incluído pela possibilidade de haver uma relação entre transações fraudulentas e a hora do dia, bem como o quão próximo está do início do mês, já que essas medidas estão relacionadas com o recebimento de salário e o próprio período de atividade do cliente (a maior parte das pessoas passa a madrugada dormindo, logo transações nesse período possuem algumas características implícitas).

Por fim, os campos *nameOrig* e *nameDest* podem ou não ser utilizados, dependendo da motivação do modelo de inteligência artificial e da sua aplicação. Várias transferências com um mesmo cliente de origem num curto período de tempo são suspeitas, assim como o recebimento de muitas transferências por um mesmo cliente de destino também são. Entretanto, o modelo de árvore de decisão utilizado lida apenas analisando uma transação isolada para definir se é ou não fraudulenta. Não há como fazer essas comparações de conjuntos sem que os nomes desses clientes tenham sido identificados durante o treinamento da inteligência artificial, ou seja, estavam presentes no conjunto de treinamento. Além disso, como estamos lidando com um problema de encontrar fraudes de maneira geral, não faz sentido aplicar a solução que estamos desenvolvendo para casos reais, com nomes de pessoas reais, e comparar seus nomes aos "nomes artificiais" presentes na tabela, como C429214117. Por esse motivo, excluiremos os campos de nome da análise, pois não agregam ao modelo.

Com isso, nosso modelo é constituído dos atributos:

1.   **step**
2.   **type**
3.   **amount**
4.   **oldbalanceOrig**
5.   **oldbalanceDest**
6.  **isFraud**

O que ajuda na construção de uma solução mais simples e rápida, que analisa menos dados e ainda assim mantém sua precisão.

### **Funcionamento de uma Árvore de Decisão**

Uma árvore de decisão consiste em uma série de perguntas a um conjunto de dados quanto aos valores de seus objetos em cada atributo. Dependendo do resultado, o conjunto se subdivide. Escolhemos as perguntas a serem feitas de modo a gerar o máximo de "ganho de informação" possível nos novos subconjuntos gerados, ou seja, obtendo uma maior proporção de objetos na classe majoritária.

Apesar de também ser usada para problemas de regressão, em que desejamos prever valores reais, árvores de decisão são popularmente utilizadas para classificação de objetos. A classificação desse problema em questão é entre "fraudulenta" e "não fraudulenta" para cada transação. Após responder uma série de perguntas e dividirmos os conjuntos em subconjuntos cada vez menores, paramos quando atingimos uma proporção suficientemente alta de objetos pertencentes à classe majoritária do conjunto. O momento em que paramos as perguntas é quando obtemos a resposta: a classe prevista para o objeto.

Várias ramificações são realizadas conforme realizamos perguntas, formando então a estrutura de uma árvore, e portanto, o caminho da raiz dessa árvore até o nó em que paramos de fazer perguntas é um dos conjuntos de características que define essa classe. Dessa forma, todo novo objeto a ser previsto entra na raiz e percorre um ramo de acordo com as suas respostas até acharmos a classe prevista.

### **Ferramenta Utilizada**

Dentre os algoritmos mais populares para a construção de árvores de decisão estão o ID3, o C4.5 e o Cart. O ID3 é um algoritmo antigo e bastante simples, sujeito a erros comuns em árvores, como overfitting aos dados devindo a grandes profundidades e uma precisão cada vez menor conforme dividimos mais os subconjuntos de forma a criar uma hierarquia em que cada nova pergunta depende das anteriores. Técnicas de poda para diminuir o tamanho da árvore são implementadas no algoritmo C4.5, que aperfeiçoa o ID3 focando somente em problemas de classificação. Além disso, realiza uma conversão da árvore de decisão para uma estrutura não hierárquica, chamada regras de decisão, que aumenta a precisão das previsões. 

Preferimos, no entanto, o algoritmo Cart, que também utiliza técnicas de poda, além de uma escolha de divisão dos nós através do índice de Gini. Seu principal benefício é a flexibilidade de lidar tanto com classificação quanto regressão. Sua popular implementação na biblioteca Scikit-Learn, facilmente integrável a outras bibliotecas de inteligência artificial em Python, foi o que nos motivou a escolhê-la.

Utilizaremos as bibliotecas Pandas, para lidar com a base de dados e análise exploratória, e Scikit-Learn para a execução do algoritmo Cart. Ao final, descreveremos os resultados pela precisão obtida, importância dos atributos (mais utilizados para dividir os dados) e uma imagem da árvore de decisão gerada.